In [26]:
%pylab inline
import pandas

Populating the interactive namespace from numpy and matplotlib


In [46]:
TRAINFILE='/home/ubuntu/data/training_input.csv'
TESTFILE='/home/ubuntu/data/testing_input.csv'
LABELFILE='/home/ubuntu/data/challenge_output_data_training_file_prediction_of_trading_activity_within_the_order_book.csv'
CHUNKSIZE=8
NCHUNKS=10
LASTROW=7
DAY=4679
HALFDAY=2239
USEFEATURES = ['ID', 'bid_1', 'ask_1', 'bid_size_1', 'ask_size_1', 'nb_trade', 'bid_entropy_1', 'ask_entropy_1', 'bid_size_2', 'ask_size_2']
NEWFEATURES = ['bid_1', 'spread', 'bid_plus_ask', 'bid_pct', 'nb_trade']+['bid_size_2', 'ask_size_2']
TRAINPICKLE='train.pkl'
TESTPICKLE='test.pkl'

In [28]:
train_all = pandas.read_csv(TRAINFILE, usecols=USEFEATURES)

In [29]:
train_all['spread'] = train_all['ask_1']-train_all['bid_1']
train_all['bid_plus_ask'] = train_all['bid_size_1']-train_all['ask_size_1']
train_all['bid_pct'] = train_all['bid_size_1']/train_all['bid_plus_ask']

In [30]:
train_diff = train_all[NEWFEATURES].diff()

In [31]:
past_diff_feat = np.hstack([train_diff.iloc[i::8, :].values for i in range(1,8)])

In [32]:
fut_diff_feat = np.hstack([train_diff.iloc[i+7::8, :].values for i in range(1,9)])
# fix end
fut_diff_feat = np.r_[fut_diff_feat, np.zeros((1, fut_diff_feat.shape[1]))]

In [33]:
pres_orig_feat = train_all.iloc[7::8][NEWFEATURES].values

In [34]:
past_orig_feat = np.hstack([train_all.iloc[i::8, 1:8].values for i in range(8)])

In [35]:
time_feat = (train_all.iloc[7::8, 0].values-1)%DAY

In [36]:
entropy_feat = train_all.iloc[7::8, -2:]

In [37]:
#train=np.c_[past_diff_feat, pres_orig_feat, time_feat]
#train=np.c_[pres_orig_feat, time_feat]
#train=pres_orig_feat
train=np.c_[past_diff_feat, fut_diff_feat, pres_orig_feat, time_feat]

In [38]:
np.save(TRAINPICKLE, train, allow_pickle=True)

In [39]:
label = pandas.read_csv(LABELFILE, sep=';')['TARGET'].values

In [40]:
import xgboost as xgb

In [41]:
dtrain = xgb.DMatrix(data=np.load(TRAINPICKLE+'.npy'), label = label)
prior=label.mean()
params={}
params['bst:eta'] = 0.1
params['bst:max_depth'] = 10
params['min_child_weight'] = 4
params['objective'] = 'binary:logistic'
params['nthread'] = 4
params['eval_metric'] = 'error'
params['learning_rate'] = 0.1
params['lambda'] = 0.1
params['base_score'] = prior
#res = xgb.cv(params, dtrain, num_boost_round=1000, verbose_eval=True, nfold=4, seed=0, early_stopping_rounds=10, show_stdv=True)

In [42]:
num_round=150
bst = xgb.train(params, dtrain, num_round)

In [47]:
test_all = pandas.read_csv(TESTFILE, usecols=USEFEATURES)

In [48]:
test_all['spread'] = test_all['ask_1']-test_all['bid_1']
test_all['bid_plus_ask'] = test_all['bid_size_1']-test_all['ask_size_1']
test_all['bid_pct'] = test_all['bid_size_1']/test_all['bid_plus_ask']

In [49]:
test_diff = test_all[NEWFEATURES].diff()

In [50]:
past_diff_feat = np.hstack([test_diff.iloc[i::8, :].values for i in range(1,8)])

In [51]:
fut_diff_feat = np.hstack([test_diff.iloc[i+7::8, :].values for i in range(1,9)])
# fix end
fut_diff_feat = np.r_[fut_diff_feat, np.zeros((1, fut_diff_feat.shape[1]))]

In [52]:
pres_orig_feat = test_all.iloc[7::8][NEWFEATURES].values

In [53]:
past_orig_feat = np.hstack([test_all.iloc[i::8, 1:8].values for i in range(8)])

In [54]:
time_feat = (test_all.iloc[7::8, 0].values-1+HALFDAY)%DAY

In [55]:
entropy_feat = test_all.iloc[7::8, -2:]

In [56]:
#train=np.c_[past_diff_feat, pres_orig_feat, time_feat]
#train=np.c_[pres_orig_feat, time_feat]
#train=pres_orig_feat
test=np.c_[past_diff_feat, fut_diff_feat, pres_orig_feat, time_feat]

In [57]:
np.save(TESTPICKLE, test, allow_pickle=True)

In [58]:
dtest = xgb.DMatrix(data=np.load(TESTPICKLE+'.npy'))

In [59]:
y_pred_test=np.round(bst.predict(dtest)).astype('int')

In [60]:
pandas.DataFrame(y_pred_test).to_csv('/tmp/pred.csv')